In [41]:
import math
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
%matplotlib inline
import pandas as pd
from keras import Sequential
from keras.layers import Dense
from keras.initializers import glorot_normal

In [42]:
#ProductId, Name, Ofert, Cost, Year, Month

#products = pd.read_csv("dataset.csv", sep="\t" )
xData = pd.read_csv("dataset.csv", usecols=['Id' , 'Precio de Mercado' , 'Temporada' , 'Descuento_Cantidad'] )
yData =  pd.read_csv("dataset.csv", usecols=['Precio Final' , 'Descuento_Bool'] )



In [43]:

#np.random.seed(0)

xArray = xData.values
yArray = yData.values 

xArray

X = np.array(xArray)
Y = np.array(yArray)


X_train = np.array( (xArray[:math.floor(xArray.shape[0] * .8) , : ])).astype(float)
Y_train = np.array( (yArray[:math.floor(xArray.shape[0] * .8), :])).astype(float)


X_test = np.array((xArray[math.floor(xArray.shape[0] * .8): ,:])).astype(float)
Y_test = np.array((yArray[math.floor(xArray.shape[0] * .8):, :])).astype(float)


# División de los datos usando train_test_split
#X_train, X_test, Y_train, Y_test = []

[[1.00000e+00 2.00000e+02 1.00000e+00 0.00000e+00]
 [1.00000e+00 2.00000e+02 2.00000e+00 0.00000e+00]
 [1.00000e+00 2.00000e+02 3.00000e+00 0.00000e+00]
 ...
 [6.00000e+00 1.31471e+03 1.00000e+01 0.00000e+00]
 [6.00000e+00 1.31471e+03 1.10000e+01 0.00000e+00]
 [6.00000e+00 1.31471e+03 1.20000e+01 0.00000e+00]]


array([[1.0000e+00, 2.0000e+02, 1.0000e+00, 0.0000e+00],
       [1.0000e+00, 2.0000e+02, 2.0000e+00, 0.0000e+00],
       [1.0000e+00, 2.0000e+02, 3.0000e+00, 0.0000e+00],
       ...,
       [5.0000e+00, 9.5488e+02, 5.0000e+00, 0.0000e+00],
       [5.0000e+00, 9.5488e+02, 6.0000e+00, 5.0000e-01],
       [5.0000e+00, 9.5488e+02, 7.0000e+00, 5.0000e-01]])

In [48]:
out_dimension = 2
nHidden = 20
nInputDimensions = X_train.shape[1]

In [49]:
modelo = Sequential()
modelo.add(Dense(nHidden, input_dim=nInputDimensions, activation='elu', \
                  kernel_initializer=glorot_normal(seed=0)))
for i in range(0, 5): # capas
    modelo.add(Dense(nHidden, activation='elu', kernel_initializer=glorot_normal(seed=0))) # agregar capa
modelo.add(Dense(out_dimension, activation='sigmoid', kernel_initializer=glorot_normal(seed=0)))

In [50]:
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
training = modelo.fit(X_train, Y_train, epochs=10)

Epoch 1/10
451/451 [==============================] - 1s 1ms/step - loss: 116818413.3980 - acc: 0.7716
Epoch 2/10
451/451 [==============================] - 0s 55us/step - loss: 116818410.4935 - acc: 1.0000
Epoch 3/10
451/451 [==============================] - 0s 46us/step - loss: 116818408.5710 - acc: 1.0000
Epoch 4/10
451/451 [==============================] - 0s 51us/step - loss: 116818413.5444 - acc: 1.0000
Epoch 5/10
451/451 [==============================] - 0s 49us/step - loss: 116818410.4404 - acc: 1.0000
Epoch 6/10
451/451 [==============================] - 0s 49us/step - loss: 116818412.0621 - acc: 1.0000
Epoch 7/10
451/451 [==============================] - 0s 49us/step - loss: 116818411.4324 - acc: 1.0000
Epoch 8/10
451/451 [==============================] - 0s 57us/step - loss: 116818412.6573 - acc: 1.0000
Epoch 9/10
451/451 [==============================] - 0s 53us/step - loss: 116818411.9911 - acc: 1.0000
Epoch 10/10
451/451 [==============================] - 0s 51us/st

In [51]:
resultados = modelo.evaluate(X_test, Y_test, batch_size=len(Y_test))
print('Accuracy:', resultados[1])

113/113 [==============================] - 0s 830us/step
Accuracy: 1.0
